视频预处理：
- 第一步：将输入的视频进行合并
- 第二步：将合并的视频按随机时长进行剪切，共剪切成n段

优化方案：
1. 在第一步的时候加入由url或者Tiktok等视频网站爬取功能
2. 在第二步的时候加入根据文本信息自动识别关联检索的功能（NLP），设计相对应的Style：Vlog、MAD、MMD等

### 零、常用方法

In [ ]:
import os
import sys
import xlwt
from moviepy.editor import VideoFileClip
 
file_dir = u"G:/视频目录/" #定义文件目录
 
class FileCheck():
 
    def __init__(self):
        self.file_dir = file_dir
    
    def get_filesize(self,filename):
        u"""
        获取文件大小（M: 兆）
        """
        file_byte = os.path.getsize(filename)
        return self.sizeConvert(file_byte)
 
    def get_file_times(self,filename):
        u"""
        获取视频时长（s:秒）
        """
        clip = VideoFileClip(filename)
        file_time = self.timeConvert(clip.duration)
        return file_time
 
    def sizeConvert(self,size):# 单位换算
        K, M, G = 1024, 1024**2, 1024**3
        if size >= G:
            return str(size/G)+'G Bytes'
        elif size >= M:
            return str(size/M)+'M Bytes'
        elif size >= K:
            return str(size/K)+'K Bytes'
        else:
            return str(size)+'Bytes'
    
    def timeConvert(self,size):# 单位换算
        M, H = 60, 60**2
        if size < M:
            return str(size)+u'秒'
        if size < H:
            return u'%s分钟%s秒'%(int(size/M),int(size%M))
        else:
            hour = int(size/H)
            mine = int(size%H/M)
            second = int(size%H%M)
            tim_srt = u'%s小时%s分钟%s秒'%(hour,mine,second)
            return tim_srt
 
    def get_all_file(self):
        u"""
        获取视频下所有的文件
        """
        for root, dirs, files in os.walk(file_dir):  
            return files #当前路径下所有非目录子文件

### 一、视频合并

In [ ]:
from moviepy.editor import *
import os
from natsort import natsorted
import json

# psutil是一个跨平台库能够轻松实现获取系统运行的进程和系统利用率（包括CPU、内存、磁盘、网络等）信息。它主要用来做系统监控，性能分析，进程管理。它实现了同等命令行工具提供的功能，如ps、top、lsof、netstat、ifconfig、who、df、kill、free、nice、ionice、iostat、iotop、uptime、pidof、tty、taskset、pmap等。目前支持32位和64位的Linux、Windows、OS X、FreeBSD和Sun Solaris等操作系统.
import psutil

# 杀死moviepy产生的特定进程
def killProcess():
    # 处理python程序在运行中出现的异常和错误
    try:
        # pids方法查看系统全部进程
        pids = psutil.pids()
        for pid in pids:
            # Process方法查看单个进程
            p = psutil.Process(pid)
            # print('pid-%s,pname-%s' % (pid, p.name()))
            # 进程名
            if p.name() == 'ffmpeg-win64-v4.1.exe':
                # 关闭任务 /f是强制执行，/im对应程序名
                cmd = 'taskkill /f /im ffmpeg-win64-v4.1.exe  2>nul 1>null'
                # python调用Shell脚本执行cmd命令
                os.system(cmd)
    except:
        pass

In [ ]:
for i in range(120):
    #提取对应视频标题的json文件路径
    myjsondirs = './video/{}/entry.json'.format(i + 1)
    #定义拼接完成后视频的标题
    vdtitle = ''
    with open(myjsondirs, 'r', encoding='UTF-8') as load_f:
        # loads方法将json格式数据转换为字典（读取文本用此法）
        load_dict = json.load(load_f)
        vdtitle = load_dict['page_data']['part']
    #视频文件夹路径
    mydirs = './video/{}/lua.flv.bili2api.80'.format(i+1)
    # 定义拼接视频的数组
    L = []
    # 访问 video 文件夹
    # root指的是当前正在遍历的这个文件夹的本身的地址，dirs是一个 list，内容是该文件夹中所有的目录的名字(不包括子目录)，files同样是 list，内容是该文件夹中所有的文件(不包括子目录)
    for root, dirs, files in os.walk(mydirs):
        # 按文件名排序
        # files.sort()
        # 自然排序法
        files = natsorted(files)
        # print(files)
        # 遍历所有文件
        for file in files:
            # os.path.splitext(“文件路径”)    分离文件名与扩展名：默认返回(fname, fextension)元组，可做分片操作
            # 如果后缀名为 .blv
            if os.path.splitext(file)[1] == '.blv':
                # .blv格式视频的完整路径
                filePath = os.path.join(root, file)
                # 读取视频到内存
                myvideo = VideoFileClip(filePath)
                # 添加到数组
                L.append(myvideo)
    # 对多个视频在时长上进行拼接
    final_clip = concatenate_videoclips(L)
    targetdir = './target/{}.mp4'.format(vdtitle)
    # 法一：生成目标视频文件方法
    # final_clip.to_videofile(targetdir, fps=24)
    #法二：最常规的生成目标视频文件方法
    final_clip.write_videofile(targetdir,fps=24, remove_temp=True)  #remove_temp=True表示生成的音频文件是临时存放的，视频生成后，音频文件会自动处理掉！若为False表示，音频文件会同时生成！
    print("{}---{}---拼接成功！".format(i + 1, vdtitle))
    killProcess()

In [9]:
from  moviepy.editor import *

L = []

if __name__ == '__main__':
    for file in os.listdir("./input/"):
        clipVideo = VideoFileClip("./input/"+file)
        L.append(clipVideo)
    
    final_clip = concatenate_videoclips(L)
    final_clip.write_videofile("./cut_result/result1.mp4",fps=24, remove_temp=False)

Moviepy - Building video result1.mp4.
MoviePy - Writing audio in result1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video result1.mp4



Moviepy - Done !
Moviepy - video ready result1.mp4


### 二、视频拆分

In [81]:
from  moviepy.editor import *

L = []
if __name__ == '__main__':
    video = VideoFileClip("result1.mp4")
    lst = list(range(int(video.duration)))
    i = 0
    
    while len(lst) != 1:       
        a = sorted(random.sample(lst,2))
        clip = VideoFileClip("result1.mp4").subclip(a[0],a[1])
#         L.append(clip)
        lst = list(set(lst) - set(list(range(a[0],a[1]))))
        clip.write_videofile("./cut_result/result"+str(i)+".mp4",fps=24, remove_temp=False)
        i += 1
    
#     final_clip = concatenate_videoclips(L)
#     final_clip.write_videofile("./cut_result/result"+i+".mp4",fps=24, remove_temp=False)       

t:   0%|                   | 2/105312 [37:24<32828:36:13, 1122.24s/it, now=None]

Moviepy - Building video ./cut_result/result0.mp4.
MoviePy - Writing audio in result0TEMP_MPY_wvf_snd.mp3



t:   0%|                   | 2/105312 [37:27<32866:19:04, 1123.53s/it, now=None]

MoviePy - Done.
Moviepy - Writing video ./cut_result/result0.mp4




t:  25%|██████▋                   | 1156/4536 [00:23<01:28, 38.17it/s, now=None]


...

t:  85%|████████████████████▍   | 16710/19680 [04:57<00:51, 57.38it/s, now=None]


t:  91%|█████████████████████▊  | 17894/19680 [05:21<00:29, 60.55it/s, now=None]


t:  98%|███████████████████████▍| 19216/19680 [05:43<00:08, 57.78it/s, now=None]


t:   0%|                 | 2/105312 [1:09:26<60940:36:43, 2083.24s/it, now=None]

Moviepy - Done !
Moviepy - video ready ./cut_result/result13.mp4


In [82]:
!shutdown -h 2

Shutdown scheduled for Tue 2021-09-14 18:27:04 CST, use 'shutdown -c' to cancel.
